In [1]:
import pandas as pd
import os
import numpy as np

path = 'C:\\Users\\Mateusz Jodlowski\\Desktop\\mgr\\t-sne'
os.chdir(path)

movies = pd.read_csv('tmdb-movies.csv', encoding = 'utf8')
movies.dropna(subset=['overview'], inplace=True)
movies.apply(lambda x: pd.lib.infer_dtype(x.values))

genres = movies['genres'].str.split('|',expand=True)
genres = genres.values
genres = pd.DataFrame(genres)
genres.columns = ['Genre 1', 'Genre 2', 'Genre 3', 'Genre 4', 'Genre 5']
genres = genres.applymap(lambda x: '' if x is None else str(x))

movies = movies.iloc[(genres.apply(lambda x: sum(x != ''), axis = 1) == 1).values]
movies=movies.rename(columns = {'genres':'Genre 1', 'overview':'Description'})
movies = movies.iloc[((movies[['Genre 1', 'Description']].isnull()).apply(sum, axis = 1) == 0).values]
movies = movies.set_index('Genre 1').join(pd.DataFrame(movies.groupby('Genre 1').size()/len(movies) > 0.1)).rename(columns = {0:'to_drop'}).reset_index()
movies = movies.iloc[movies['to_drop'].values]
movies = movies.drop(['to_drop'], axis = 1)

C:\Users\Mateusz Jodlowski\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: pandas.lib is deprecated and will be removed in a future version.
You can access infer_dtype as pandas.api.types.infer_dtype
  # Remove the CWD from sys.path while we load stuff.


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

stop = stopwords.words('english')

docs = movies['Description']

docs = docs.apply(lambda x: x.translate(str.maketrans({key: None for key in string.punctuation})))
docs = docs.apply(lambda x: x.lower())

docs = docs.apply(lambda x: [x for x in x.split() if x not in stop])
docs = docs.apply(lambda x: ' '.join(x))

porter = PorterStemmer()

docs = [porter.stem(word) for word in docs]

vec = CountVectorizer()

X = vec.fit_transform(docs)
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

print(df)

      007  10  100  10000  100000  1000000  100km  100point  10episode  \
0       0   0    0      0       0        0      0         0          0   
1       0   0    0      0       0        0      0         0          0   
2       0   0    0      0       0        0      0         0          0   
3       0   0    0      0       0        0      0         0          0   
4       0   0    0      0       0        0      0         0          0   
5       0   0    0      0       0        0      0         0          0   
6       0   0    0      0       0        0      0         0          0   
7       0   0    0      0       0        0      0         0          0   
8       0   0    0      0       0        0      0         0          0   
9       0   0    0      0       0        0      0         0          0   
10      0   0    0      0       0        0      0         0          0   
11      0   0    0      0       0        0      0         0          0   
12      0   0    0      0       0     

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
import string
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer

stop = stopwords.words('english')

docs = movies['Description']

docs = docs.apply(lambda x: x.translate(str.maketrans({key: None for key in string.punctuation})))
docs = docs.apply(lambda x: x.lower())

docs = docs.apply(lambda x: [x for x in x.split() if x not in stop])
docs = docs.apply(lambda x: ' '.join(x))

lancaster = LancasterStemmer()

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

# docs = [lancaster.stem(word) for word in docs]

docs = [wordnet_lemmatizer.lemmatize(word) for word in docs]

vec = CountVectorizer()

X = vec.fit_transform(docs)
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

print(df)

      007  10  100  10000  100000  1000000  100km  100point  10episode  \
0       0   0    0      0       0        0      0         0          0   
1       0   0    0      0       0        0      0         0          0   
2       0   0    0      0       0        0      0         0          0   
3       0   0    0      0       0        0      0         0          0   
4       0   0    0      0       0        0      0         0          0   
5       0   0    0      0       0        0      0         0          0   
6       0   0    0      0       0        0      0         0          0   
7       0   0    0      0       0        0      0         0          0   
8       0   0    0      0       0        0      0         0          0   
9       0   0    0      0       0        0      0         0          0   
10      0   0    0      0       0        0      0         0          0   
11      0   0    0      0       0        0      0         0          0   
12      0   0    0      0       0     

In [4]:
df = df.iloc[:,((df>0).apply(sum) > 1).values]
import re
df = df.iloc[:, pd.Series(df.columns).apply(lambda x: re.match("^[0-9]", x) is None).values]
df

,aaron,abandoned,abby,abbys,abcs,abducted,abilities,ability,able,aboard,...,zack,zana,zero,zisman,zombie,zombies,zone,œi,œthe,œtwo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
dtm = movies['Genre 1'].reset_index()[movies['Genre 1'].reset_index().columns.difference(['index'])]
groups_filter = pd.concat([dtm, df], axis=1)
groups_filter = groups_filter.groupby('Genre 1').sum().reset_index()[df.columns]
df = df.iloc[:, (groups_filter.apply(lambda x: sum(x != 0)) == 1).values]

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(df)
tfidf = pd.DataFrame(tfidf.toarray())
tfidf.columns = df.columns.difference(['Genre 1'])
tfidf

,abcs,accent,accessories,accounted,ace,achieve,achmed,acquaintance,activism,adapt,...,wreaking,wrestling,yearns,yorks,youll,youthful,zac,zack,zisman,zombies
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [7]:
tfidf = tfidf.reset_index()[tfidf.reset_index().columns.difference(['index'])]
first_class = pd.concat([dtm, tfidf], axis=1)
first_class

,Genre 1,abcs,accent,accessories,accounted,ace,achieve,achmed,acquaintance,activism,...,wreaking,wrestling,yearns,yorks,youll,youthful,zac,zack,zisman,zombies
0,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,Comedy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [8]:
first_class.groupby('Genre 1').size()

Genre 1
Comedy         711
Documentary    312
Drama          712
Horror         253
dtype: int64

In [9]:
from sklearn.manifold import TSNE
import random 
#random.seed(1993)
feed = first_class[first_class.columns.difference(['Genre 1'])]
first_class_embedded = TSNE(n_components=2, perplexity = 5).fit_transform(feed)
first_class_embedded = pd.DataFrame(first_class_embedded)
first_class_embedded['target'] = first_class['Genre 1']
first_class_embedded

,0,1,target
0,-66.243858,29.711699,Comedy
1,-40.518066,-10.943316,Comedy
2,-0.126909,5.602508,Comedy
3,-66.716164,-18.097933,Comedy
4,-3.395150,5.959115,Comedy
5,0.787545,6.288303,Comedy
6,-1.522356,6.484133,Comedy
7,0.860110,4.699847,Comedy
8,-21.527298,-41.712936,Comedy
9,-2.783078,7.706259,Comedy


In [10]:
import matplotlib.pyplot as plt
import seaborn
seaborn.set(style='ticks')
import pylab
first_class_embedded.columns = ['x', 'y', 'target']
fg = seaborn.FacetGrid(data=first_class_embedded, hue='target')
fig = fg.map(plt.scatter, 'x', 'y').add_legend()
pylab.savefig('plots\\rerun_6.png')

In [21]:
y = first_class_embedded['target']
X = first_class_embedded[first_class_embedded.columns.difference(['target'])]

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1993)

clf.fit(X_train, y_train)

(clf.predict(X_test) == y_test).mean()

In [42]:
y = first_class['Genre 1']
X = first_class[first_class.columns.difference(['Genre 1'])]

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1993)

clf.fit(X_train, y_train)

(clf.predict(X_test) == y_test).mean()

0.82160804020100497

In [44]:
import numpy as np
from mogutda import SimplicialComplex

torus_sc = [(1,2,4), (4,2,5), (2,3,5), (3,5,6), (5,6,1), (1,6,2), (6,7,2), (7,3,2),
            (1,3,4), (3,4,6), (4,6,7), (4,5,7), (5,7,1), (7,3,1)]
torus_c = SimplicialComplex(simplices=torus_sc)

In [45]:
print(torus_c.betti_number(0))   # print 1
print(torus_c.betti_number(1))   # print 2
print(torus_c.betti_number(2)) 

1
2
1


0.89195979899497491